In [1]:
import requests
import json  ### Intentar cambiar el formato JSON por uno más específico de Python
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Aceso a todas las variantes de un gen a través de la API de Ensembl

In [2]:
def acceso_variantes_ensembl (identificador_gen, especie):
    """ 
    DESCRIPCIÓN DE LA FUNCIÓN"""
    
    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/overlap/id/{identificador_gen}?feature=variation;species={especie}"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variantes = solicitud.json()
    
    return variantes



## Obtención de las variantes patogénicas a partir de las variantes que hemos obtenido de la API de Ensembl

In [3]:
def obtener_variantes_patogenicas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_patogenicas = []

    for var in variantes:
        if "clinical_significance" in var and "pathogenic" in var["clinical_significance"]:
            variantes_patogenicas.append(var)


    return variantes_patogenicas
            

## Obtención de las variantes benignas a partir de las variantes que hemos obtenido de la API de Ensembl

In [4]:
def obtener_variantes_benignas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_benignas = []

    for var in variantes:
        if "clinical_significance" in var and "benign" in var["clinical_significance"]:
            variantes_benignas.append(var)


    return variantes_benignas
            

In [223]:
variantes_benignas = obtener_variantes_benignas ("ENST00000269305", "human")

In [224]:
for i in variantes_benignas[:5]:
    print (i)

{'assembly_name': 'GRCh38', 'clinical_significance': ['uncertain significance', 'benign', 'risk factor'], 'id': 'rs78378222', 'start': 7668434, 'consequence_type': '3_prime_UTR_variant', 'feature_type': 'variation', 'source': 'dbSNP', 'alleles': ['T', 'G'], 'strand': 1, 'seq_region_name': '17', 'end': 7668434}
{'start': 7668539, 'id': 'rs114831472', 'clinical_significance': ['benign', 'likely benign'], 'assembly_name': 'GRCh38', 'end': 7668539, 'strand': 1, 'seq_region_name': '17', 'source': 'dbSNP', 'feature_type': 'variation', 'alleles': ['G', 'A', 'C'], 'consequence_type': '3_prime_UTR_variant'}
{'id': 'rs17884306', 'clinical_significance': ['benign'], 'assembly_name': 'GRCh38', 'start': 7668783, 'seq_region_name': '17', 'strand': 1, 'feature_type': 'variation', 'alleles': ['C', 'A', 'G', 'T'], 'source': 'dbSNP', 'consequence_type': '3_prime_UTR_variant', 'end': 7668783}
{'start': 7668836, 'clinical_significance': ['benign'], 'assembly_name': 'GRCh38', 'id': 'rs200378797', 'end': 76

In [225]:
variantes_patogenicas = obtener_variantes_patogenicas ("ENST00000269305", "human")

In [226]:
for i in variantes_patogenicas[:5]:
    print (i)

{'start': 7669666, 'id': 'rs730882017', 'assembly_name': 'GRCh38', 'clinical_significance': ['pathogenic'], 'end': 7669666, 'seq_region_name': '17', 'strand': 1, 'consequence_type': 'frameshift_variant', 'feature_type': 'variation', 'alleles': ['C', '-'], 'source': 'dbSNP'}
{'end': 7669677, 'seq_region_name': '17', 'strand': 1, 'consequence_type': 'frameshift_variant', 'feature_type': 'variation', 'source': 'dbSNP', 'alleles': ['TTTTT', 'TTTT'], 'start': 7669673, 'id': 'rs2150988776', 'assembly_name': 'GRCh38', 'clinical_significance': ['uncertain significance', 'pathogenic']}
{'assembly_name': 'GRCh38', 'clinical_significance': ['likely pathogenic', 'pathogenic'], 'id': 'rs876658982', 'start': 7669691, 'consequence_type': 'splice_acceptor_variant', 'source': 'dbSNP', 'feature_type': 'variation', 'alleles': ['C', 'A', 'T'], 'seq_region_name': '17', 'strand': 1, 'end': 7669691}
{'end': 7669692, 'strand': 1, 'seq_region_name': '17', 'consequence_type': 'splice_acceptor_variant', 'feature

## Obtención de la localización de las variantes

In [227]:
def obtener_localizacion_variantes (variantes):

    diccionario_variantes = {}

    for var in variantes:
        inicio = var["start"]
        final = var["end"]
        longitud = final - inicio
        diccionario_variantes[var["id"]] = [inicio, final, longitud]
    
    return diccionario_variantes
        

## Obtención de los exones

In [228]:
def obtener_exones_gen (identificador_transcrito, especie):
    
    rest_api = "https://rest.ensembl.org"
    extension_exones = f"/lookup/id/{identificador_transcrito}?expand=1&species={especie}"

    solicitud = requests.get(rest_api+extension_exones, headers={"Content-Type" : "application/json"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    exones = solicitud.json()

    return exones

    

## Obtención de la secuencia de los exones en formato fasta

In [323]:
def obtener_secuencia_exones_fasta (identificador_transcrito, especie, nombre_archivo_fasta):
    exones = obtener_exones_gen (identificador_transcrito, especie)

    lista_secuencias = []

    for ex in exones["Exon"]:
        identificador_exon = ex["id"]

        rest_api = "https://rest.ensembl.org"
        extension_secuencia_exon = f"/sequence/id/{identificador_exon}?content-type=text/x-fasta"

        solicitud2 = requests.get(rest_api+extension_secuencia_exon, headers={ "Content-Type" : "application/json"})

        if not solicitud2.ok:
            solicitud2.raise_for_status()
            sys.exit()

        exon = solicitud2.json()
        
        if ex["strand"] == -1:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]).complement())
            ex["strand"] = 1
        else:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]).complement())
            
        secuencia_fasta.id = exon["id"]
        secuencia_fasta.name = exon["id"]
        secuencia_fasta.description = exon["desc"]
        lista_secuencias.append(secuencia_fasta)

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    return exones["Exon"]

    print ("Se han guardado las secuencias correctamente")

In [324]:
a = obtener_secuencia_exones_fasta ("ENST00000269305", "human", "data/hola.fasta")

In [325]:
for i in a:
    print (i)

{'seq_region_name': '17', 'strand': 1, 'species': 'homo_sapiens', 'object_type': 'Exon', 'version': 2, 'assembly_name': 'GRCh38', 'end': 7687490, 'start': 7687377, 'db_type': 'core', 'id': 'ENSE00003753508'}
{'assembly_name': 'GRCh38', 'version': 1, 'id': 'ENSE00004023728', 'db_type': 'core', 'start': 7676521, 'end': 7676622, 'strand': 1, 'seq_region_name': '17', 'object_type': 'Exon', 'species': 'homo_sapiens'}
{'assembly_name': 'GRCh38', 'version': 1, 'id': 'ENSE00002419584', 'db_type': 'core', 'start': 7676382, 'end': 7676403, 'strand': 1, 'seq_region_name': '17', 'object_type': 'Exon', 'species': 'homo_sapiens'}
{'object_type': 'Exon', 'species': 'homo_sapiens', 'strand': 1, 'seq_region_name': '17', 'db_type': 'core', 'id': 'ENSE00003625790', 'start': 7675994, 'end': 7676272, 'assembly_name': 'GRCh38', 'version': 1}
{'object_type': 'Exon', 'species': 'homo_sapiens', 'strand': 1, 'seq_region_name': '17', 'start': 7675053, 'db_type': 'core', 'id': 'ENSE00003518480', 'end': 7675236, '

## Obtención de la localización de los exones

In [311]:
def obtener_localizacion_exones (exones):
    
    diccionario_exones = {}

    for exon in exones["Exon"]:
        inicio = exon["start"]
        final = exon["end"]
        longitud = final - inicio
        diccionario_exones[exon["id"]] = [inicio, final, longitud]
    
    return diccionario_exones
        
    

In [312]:
obtener_localizacion_exones (exones)

{'ENSE00003753508': [7687377, 7687490, 113],
 'ENSE00004023728': [7676521, 7676622, 101],
 'ENSE00002419584': [7676382, 7676403, 21],
 'ENSE00003625790': [7675994, 7676272, 278],
 'ENSE00003518480': [7675053, 7675236, 183],
 'ENSE00003723991': [7674859, 7674971, 112],
 'ENSE00003712342': [7674181, 7674290, 109],
 'ENSE00003725258': [7673701, 7673837, 136],
 'ENSE00003786593': [7673535, 7673608, 73],
 'ENSE00003545950': [7670609, 7670715, 106],
 'ENSE00004023724': [7668421, 7669690, 1269]}

## Obtención de las variantes que se encuentran dentro de los exones

In [313]:
loc_variantes_patogenicas = obtener_localizacion_variantes(variantes_patogenicas)

loc_exones = obtener_localizacion_exones (exones)

In [330]:
secuencia = SeqIO.parse ("data/hola.fasta", "fasta")


In [331]:
a = next(secuencia)
print (a.seq)

GAGTTTTCAGATCTCGGTGGCAGGTCCCTCGTCCATCGACGACCCGAGGCCCCTGTGAAACGCAAGCCCGACCCTCGCACGAAAGGTGCTGCCACTGTGCGAAGGGACCTAACC


In [295]:
b = a.seq.complement()
print (b)

GTCGGTCTGACGGAAGGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATCGCAGCTCGGGGGAGACTCAGTCCTTTGTAAAAGTCTGGATACCTTTGA


In [290]:
exon1 = exones["Exon"][1]

inicio_exon = exon1["start"]
final_exon = exon1["end"]

diccionario_variantes = {}

for i in variantes_patogenicas:
    inicio_variante = i["start"]

    if inicio_exon <= inicio_variante <= final_exon:
        diccionario_variantes[i["id"]] = i

print (diccionario_variantes)



    

{'rs1131691018': {'start': 7676537, 'id': 'rs1131691018', 'assembly_name': 'GRCh38', 'clinical_significance': ['pathogenic'], 'end': 7676540, 'seq_region_name': '17', 'strand': 1, 'consequence_type': 'frameshift_variant', 'source': 'dbSNP', 'feature_type': 'variation', 'alleles': ['AAAA', 'AAA']}, 'rs876658627': {'consequence_type': 'frameshift_variant', 'feature_type': 'variation', 'source': 'dbSNP', 'alleles': ['TTT', 'TT'], 'strand': 1, 'seq_region_name': '17', 'end': 7676545, 'assembly_name': 'GRCh38', 'clinical_significance': ['pathogenic'], 'id': 'rs876658627', 'start': 7676543}, 'rs1555526997': {'end': 7676550, 'alleles': ['A', '-'], 'feature_type': 'variation', 'source': 'dbSNP', 'consequence_type': 'frameshift_variant', 'strand': 1, 'seq_region_name': '17', 'start': 7676550, 'clinical_significance': ['pathogenic'], 'assembly_name': 'GRCh38', 'id': 'rs1555526997'}, 'rs1555527002': {'end': 7676561, 'strand': 1, 'seq_region_name': '17', 'alleles': ['GGGGG', 'GGGGGG'], 'feature_ty

## Obtención de la secuencia de las variantes, tanto patogénicas, como benignas

Como al utilizar la API de Ensembl para obtener las variantes no obtenemos la secuencia completa del gen con estas variaciones, sino que solo obtenemos los nucleótidos que han sufrido la variación. Para obtener dichas secuencias con las variaciones es necesario seguir dos pasos:

1. Obtenemos la secuencia de referencia del gen.
   
3. Aplicamos las variaciones que hemos obtenido a través de la API sobre la secuencia de referencia del gen. 

In [ ]:
def aplicar_variante 

In [218]:
def aplicar_variante(secuencia_exon, variante):
    """
    Función que aplica una variante a la secuencia de un exon.

    Args:
    - secuencia_exon (str): Secuencia original del exon en formato de cadena.
    - variante (dict): Información de la variante en formato de diccionario.

    Returns:
    - str: Secuencia modificada del exon con la variante aplicada.
    """
    start = variante['start'] - 1  # Ajuste a base 0 para índices de cadena
    end = variante['end']
    alleles = variante['alleles']

    # Verificar el tipo de variante (asumiendo que es una inserción o deleción simple)
    if len(alleles[0]) > len(alleles[1]):
        # Deleción (eliminar nucleótidos)
        nueva_secuencia = secuencia_exon[:start] + alleles[1] + secuencia_exon[end:]
    elif len(alleles[0]) < len(alleles[1]):
        # Inserción (añadir nucleótidos)
        nueva_secuencia = secuencia_exon[:start] + alleles[1] + secuencia_exon[start:]
    else:
        # Reemplazo (sustituir nucleótidos)
        nueva_secuencia = secuencia_exon[:start] + alleles[1] + secuencia_exon[end:]

    return nueva_secuencia


# Ejemplo de uso:

# Suponiendo que tienes las secuencias de los exones en un diccionario
secuencias_exones = {
    'Exon1': "ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC",
    'Exon2': "TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT"
}

# Suponiendo que tienes el diccionario de variantes
variantes = {
    'rs1131691018': {
        'consequence_type': 'frameshift_variant',
        'alleles': ['AAAA', 'AAA'],
        'clinical_significance': ['pathogenic'],
        'end': 7676540,
        'source': 'dbSNP',
        'feature_type': 'variation',
        'strand': 1,
        'seq_region_name': '17',
        'assembly_name': 'GRCh38',
        'start': 7676537,
        'id': 'rs1131691018'
    }
}

# Obtener la secuencia modificada del Exon1 con la variante rs1131691018
exon_modificado = aplicar_variante(secuencias_exones['Exon1'], variantes['rs1131691018'])

print(f"Secuencia modificada del Exon1: {exon_modificado}")


Secuencia modificada del Exon1: ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCAAA


In [219]:
print ("ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCAAA" == "ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC")

False


In [222]:
asq = "ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC"
asf = "ATGCGATCGATCGATCGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCAAA"

print (len(asf))

64


## Obtención de la secuencia de las variantes patogénicas en formato fasta

Como las variantes que hemos obtenido en un primer paso de obtención de variantes sabemos que corresponden a humanos, no necesitamso especificar la especie en este caso.

In [11]:
variant_id = "rs1333049"

# URL del punto final para obtener detalles de la variante
url = f"https://rest.ensembl.org/variation/human/{variant_id}?content-type=application/json"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Imprimir la estructura de la respuesta JSON para inspección
    print(data)

    # Verificar y procesar los datos para crear la secuencia FASTA
    if 'mappings' in data and len(data['mappings']) > 0:
        seq_region_name = data['mappings'][0]['seq_region_name']
        start = data['mappings'][0]['start']
        end = data['mappings'][0]['end']
        if 'alleles' in data:
            alleles = data['alleles']
            for allele in alleles:
                if 'allele' in allele:
                    allele_string = allele['allele']
                    # Crear la secuencia FASTA
                    fasta_header = f">{variant_id} {seq_region_name}:{start}-{end}"
                    fasta_sequence = f"{allele_string}"
                    fasta_output = f"{fasta_header}\n{fasta_sequence}"
                    
                    # Imprimir la secuencia en formato FASTA
                    print(fasta_output)
        else:
            print("No 'alleles' key found in the data.")
    else:
        print("No 'mappings' key found in the data or mappings list is empty.")
else:
    print(f"Error: {response.status_code}")

{'var_class': 'SNP', 'evidence': ['Frequency', '1000Genomes', 'Cited', 'Phenotype_or_Disease', 'TOPMed', 'gnomAD'], 'name': 'rs1333049', 'most_severe_consequence': 'intron_variant', 'ambiguity': 'V', 'minor_allele': None, 'source': 'Variants (including SNPs and indels) imported from dbSNP', 'clinical_significance': ['risk factor'], 'MAF': None, 'mappings': [{'coord_system': 'chromosome', 'seq_region_name': '9', 'end': 22125504, 'ancestral_allele': 'C', 'assembly_name': 'GRCh38', 'start': 22125504, 'allele_string': 'G/A/C', 'location': '9:22125504-22125504', 'strand': 1}], 'synonyms': ['PA166157726', 'VCV000812642', 'RCV001003460', 'rs58844516', 'rs59077428', 'rs17761501']}
No 'alleles' key found in the data.


In [13]:
variant_id = "rs1555523630"

# URL del punto final para obtener detalles de la variante
url = f"https://rest.ensembl.org/variation/human/{variant_id}?content-type=application/json"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Verificar y procesar los datos para crear la secuencia FASTA
    if 'mappings' in data and len(data['mappings']) > 0:
        mapping = data['mappings'][0]
        seq_region_name = mapping['seq_region_name']
        start = mapping['start']
        end = mapping['end']
        allele_string = mapping['allele_string']

        # Crear la secuencia FASTA para cada alelo
        alleles = allele_string.split('/')
        for allele in alleles:
            fasta_header = f">{variant_id} {seq_region_name}:{start}-{end}"
            fasta_sequence = allele
            fasta_output = f"{fasta_header}\n{fasta_sequence}"
            
            # Imprimir la secuencia en formato FASTA
            print(fasta_output)
else:
    print(f"Error: {response.status_code}")

>rs1555523630 17:7668203-7668204
CC
>rs1555523630 17:7668203-7668204
CCC


In [11]:
def get_sequence_with_variants(gene_id, variant_ids):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{gene_id}?type=genomic;var_synonyms={','.join(variant_ids)};content-type=text/x-fasta"
    
    response = requests.get(server + ext, headers={ "Content-Type" : "text/x-fasta"})
    
    if not response.ok:
        response.raise_for_status()
    
    return response.text

# Ejemplo de uso
gene_id = "ENSG00000012048"  # ID del gen BRCA1
variant_ids = ["rs80357713"]

sequence_with_variants = get_sequence_with_variants(gene_id, variant_ids)
print(sequence_with_variants)

>ENSG00000012048.26 chromosome:GRCh38:17:43044295:43170245:-1
...
AAACACTTCCA



In [77]:
def obtener_secuencia_variante (identificador_variante):

    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variante_informacion = solicitud.json()
    print (variante_informacion)
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

    localizacion = variante_informacion["mappings"][0]
    cromosoma = localizacion["seq_region_name"]
    inicio_variante = localizacion["start"]
    fin_variante = localizacion["end"]

    flanco_secuencia = 50
    inicio_secuencia = inicio_variante - flanco_secuencia
    fin_secuencia = fin_variante + flanco_secuencia

    extension_secuencia_variantes = f"/sequence/region/human/{cromosoma}:{inicio_secuencia}..{fin_secuencia}:1?content-type=text/plain"
    solicitud2 = requests.get(rest_api+extension_secuencia_variantes, headers={"Content-Type": "text/plain"})

    if not solicitud2.ok:
        solicitud2.raise_for_status()
        sys.exit()

    secuencia_variante = solicitud2.text
        
    print (secuencia_variante)
    return secuencia_variante

        
    

In [79]:
obtener_secuencia_variante ("rs587781664")

{'evidence': ['Cited', 'Phenotype_or_Disease'], 'ambiguity': 'N', 'minor_allele': None, 'source': 'Variants (including SNPs and indels) imported from dbSNP', 'clinical_significance': ['not provided', 'likely pathogenic', 'pathogenic'], 'MAF': None, 'name': 'rs587781664', 'most_severe_consequence': 'splice_acceptor_variant', 'mappings': [{'allele_string': 'T/A/C/G', 'end': 7669692, 'strand': 1, 'assembly_name': 'GRCh38', 'location': '17:7669692-7669692', 'ancestral_allele': 'T', 'coord_system': 'chromosome', 'seq_region_name': '17', 'start': 7669692}], 'var_class': 'SNP', 'synonyms': ['VCV000635383', 'RCV000786810', 'VCV000141332', 'RCV000206779', 'RCV002288631', 'RCV000129814', 'VCV001264355', 'RCV000213070', 'RCV001667873', 'TP53_g.17908A>G']}
ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA


'ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA'

In [76]:

sec1 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
sec2 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
print (sec1 == sec2)

True


In [52]:
def variantes_region (identificador_gen, especie, cromosoma_gen, pos_inicio, pos_fin):
    variantes_patogenicas = obtener_variantes_patogenicas (identificador_gen, especie)
    diccionario_variantes = {}

    for i in variantes_patogenicas:
        rest_api = "https://rest.ensembl.org"
        extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

        solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


        if not solicitud.ok:
            solicitud.raise_for_status()
            sys.exit()

        variante_informacion = solicitud.json()
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

        localizacion = variante_informacion["mappings"][0]
        cromosoma = localizacion["seq_region_name"]
        inicio_variante = localizacion["start"]
        fin_variante = localizacion["end"]


        if cromosoma == cromosoma_gen and inicio_variante > pos_inicio and inicio_variante < pos_fin and fin_variante > pos_inicio and fin_variante < pos_fin:
            diccionario_variantes[i["id"]] = i

    return diccionario_variantes


        

    
    

In [66]:
def guardar_fasta (identificador_variante):
    x = SeqRecord(Seq((obtener_secuencia_variante(identificador_variante))))
    SeqIO.write (x, "secuencia.fasta", "fasta")
    

In [67]:
def obtener_secuencia_gen(id_gen):
    rest_api = "https://rest.ensembl.org"
    extension_gen = f"/sequence/id/{id_gen}?content-type=text/plain"
    
    solicitud = requests.get(rest_api + extension_gen, headers={"Content-Type": "text/plain"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        return None

    secuencia_gen = solicitud.text

In [73]:
x = obtener_secuencia_gen ("ENSG00000141510")